In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# 📦 Paramètres
IMG_SIZE = 299  # Xception attend des images 299x299
BATCH_SIZE = 32
EPOCHS = 10
DATA_DIR = './stanford_dogs/Images'  # Dossier contenant les sous-dossiers par race

# 🔄 Prétraitement des images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# 🧠 Chargement du modèle Xception sans la dernière couche
base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# 🔒 On gèle les couches du modèle pré-entraîné
base_model.trainable = False

# 🏗️ Ajout des couches de classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# ⚙️ Compilation
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 🚀 Entraînement
model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

# 💾 Sauvegarde du modèle
model.save('xception_dog_breed_classifier.h5')

Found 16508 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


c:\Users\lea\Downloads\Refactoring Kata\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 2576s 5s/step - accuracy: 0.8103 - loss: 0.7063 - val_accuracy: 0.8539 - val_loss: 0.4928
Epoch 2/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 4536s 9s/step - accuracy: 0.8726 - loss: 0.3897 - val_accuracy: 0.8524 - val_loss: 0.4905
Epoch 3/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 2039s 4s/step - accuracy: 0.8911 - loss: 0.3270 - val_accuracy: 0.8681 - val_loss: 0.4335
Epoch 4/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1419s 3s/step - accuracy: 0.9050 - loss: 0.2841 - val_accuracy: 0.8708 - val_loss: 0.4669
Epoch 5/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1112s 2s/step - accuracy: 0.9110 - loss: 0.2611 - val_accuracy: 0.8551 - val_loss: 0.4983
Epoch 6/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1176s 2s/step - accuracy: 0.9197 - loss: 0.2268 - val_accuracy: 0.8644 - val_loss: 0.4891
Epoch 7/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 3142s 6s/step - accuracy: 0.9265 - loss: 0.2017 - val_accuracy: 0.8585 - val_loss: 0.5414
Epoch 8/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1372s 3s/step - accuracy: 0.9325 - loss: 0.1930 - 

In [2]:
import numpy as np
loss, accuracy = model.evaluate(val_generator)
print(f"Accuracy: {accuracy:.2%}")
print(f"Loss: {loss:.4f}")
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes
class_labels = list(val_generator.class_indices.keys())
    


128/128 ━━━━━━━━━━━━━━━━━━━━ 800s 6s/step - accuracy: 0.8649 - loss: 0.5732
Accuracy: 86.49%
Loss: 0.5732
128/128 ━━━━━━━━━━━━━━━━━━━━ 769s 6s/step
